In [63]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [78]:
regions_dict = {
    "Central": [
        "CENTRAL AREA", "BISHAN", "BUKIT MERAH", "BUKIT TIMAH", "DOWNTOWN CORE", "GEYLANG", "KALLANG",
        "MARINA EAST", "MARINA SOUTH", "MARINE PARADE", "MUSEUM", "NEWTON", "NOVENA", "ORCHARD", "OUTRAM",
        "QUEENSTOWN", "RIVER VALLEY", "ROCHOR", "SINGAPORE RIVER", "SOUTHERN ISLANDS", "STRAITS VIEW", "TANGLIN", "TOA PAYOH"
    ],
    "East": ["BEDOK", "CHANGI", "CHANGI BAY", "PASIR RIS", "PAYA LEBAR", "TAMPINES"],
    "North": ["CENTRAL WATER CATCHMENT", "LIM CHU KANG", "MANDAI", "SEMBAWANG", "SIMPANG", "SUNGEI KADUT", "WOODLANDS", "YISHUN"],
    "North-East": ["ANG MO KIO", "HOUGANG", "NORTH-EASTERN ISLANDS", "PUNGGOL", "SELETAR", "SENGKANG", "SERANGOON"],
    "West": ["BOON LAY", "BUKIT BATOK", "BUKIT PANJANG", "CHOA CHU KANG", "CLEMENTI", "JURONG EAST", "JURONG WEST",
             "PIONEER", "TENGAH", "TUAS", "WESTERN ISLANDS", "WESTERN WATER CATCHMENT"]
}

df_carpark = pd.read_csv("HDBCarparkInformation.csv")

# Function to assign region based on address
def get_region(address):
    for region, areas in regions_dict.items():
        if any(area in address for area in areas):
            return region
    return 'Unknown'

# Apply the function to create the 'region' column
df_carpark['region'] = df_carpark['address'].apply(get_region)
df_carpark.to_csv('Checkmark1.csv')

In [64]:
# Load the Excel file
file_name = "SES_2024_tidy.xlsx.coredownload.xlsx"
sheet_name = "T3.5"

# Read the data from the Excel file
df = pd.read_excel(file_name, sheet_name=sheet_name)
df = df.fillna(0)

# Treat the random 's' as 0
df['Kwh_Per_Acc_Cleaned'] = df['Kwh Per Acc'].apply(lambda x: 0 if isinstance(x, str) and 's' in x else x) 

# Filter the data for the year 2024 and 'Dwelling Type' as 'overall'
filtered_data = df[(df['Year'] == 2024) & (df['Dwelling Type'] == 'Overall')]

# Calculate total consumption per region
regions = ['Central Region', 'East Region', 'North East Region', 'North Region', 'West Region']
demand_kW_per_region = {
    region: filtered_data[filtered_data['Region'] == region]['Kwh_Per_Acc_Cleaned'].astype(float).sum()
    for region in regions
}

demand_kW_per_region

{'Central Region': 88877.8,
 'East Region': 27358.199999999997,
 'North East Region': 23995.5,
 'North Region': 23482.0,
 'West Region': 19945.699999999997}

In [65]:
# Define other parameters
types_of_panels = ['Monocrystalline', 'Polycrystalline', 'Thin-film']
efficiency = {'Monocrystalline': 0.15, 'Polycrystalline': 0.13, 'Thin-film': 0.07}
fixed_cost = {'Monocrystalline': 0.75, 'Polycrystalline': 0.70, 'Thin-film': 0.30}
install_cost = {'Monocrystalline': 300, 'Polycrystalline': 250, 'Thin-film': 200}
maintenance_cost_percentage = 0.01
panel_area_ft2 = 225 # Assuming the size of the panels are 15x15
panel_area_m2 = panel_area_ft2 * 0.092903
available_space_m2 = 2559.5
max_power_per_panel_kW = 400  # Maximum power per panel in kW
budget_limit = 1_000_000  # Budget limit in dollars

In [66]:
# Create a new model
model = gp.Model("solar_optimization")

# Decision variables
x = model.addVars(regions, types_of_panels, vtype=GRB.INTEGER, name="x")
y = model.addVars(regions, types_of_panels, vtype=GRB.BINARY, name="y")

In [67]:
# Objective function: Minimize total cost
model.setObjective(
    gp.quicksum(
        (fixed_cost[j] + install_cost[j] + maintenance_cost_percentage * (fixed_cost[j] + install_cost[j])) * x[i, j]
        for i in regions for j in types_of_panels
    ),
    GRB.MINIMIZE
)

<h3>Can budget by maximum number of panels you want to build if you do this way it's 1000 in the last line</h3>

In [68]:
# # Constraints
# for i in regions:
#     # Demand constraint with maximum power per panel
#     model.addConstr(
#         gp.quicksum(efficiency[j] * max_power_per_panel_kW * x[i, j] for j in types_of_panels) >= demand_kW_per_region[i],
#         name=f"demand_{i}"
#     )

#     # Space constraint
#     model.addConstr(
#         gp.quicksum(x[i, j] * panel_area_m2 for j in types_of_panels) <= available_space_m2,
#         name=f"space_{i}"
#     )

#     # Installation constraints
#     for j in types_of_panels:
#         model.addConstr(x[i, j] <= 1000 * y[i, j], name=f"install_{i}_{j}")

<h3>Or if you do this way then you budget by the money set in the cell above. For now I put $1 mil</h3>

In [69]:
# Constraints
for i in regions:
    # Demand constraint with maximum power per panel
    model.addConstr(
        gp.quicksum(efficiency[j] * max_power_per_panel_kW * x[i, j] for j in types_of_panels) >= demand_kW_per_region[i],
        name=f"demand_{i}"
    )

    # Space constraint
    model.addConstr(
        gp.quicksum(x[i, j] * panel_area_m2 for j in types_of_panels) <= available_space_m2,
        name=f"space_{i}"
    )

    # Installation constraints
    for j in types_of_panels:
        model.addConstr(x[i, j] <= 1000 * y[i, j], name=f"install_{i}_{j}")

# Budget constraint
model.addConstr(
    gp.quicksum(
        (fixed_cost[j] + install_cost[j] + maintenance_cost_percentage * (fixed_cost[j] + install_cost[j])) * x[i, j]
        for i in regions for j in types_of_panels
    ) <= budget_limit,
    name="budget"
)

<gurobi.Constr *Awaiting Model Update*>

In [70]:
# Set the txt file to see the logs
model.setParam('LogFile', 'gurobi_log.txt')
model.setParam('OutputFlag', 1) 

# Optimize the model
model.optimize()

# In case we mess up and we wanna see the variables 
if model.status == GRB.INFEASIBLE:
    model.computeIIS()
    model.write("model.ilp")

Set parameter LogFile to value "gurobi_log.txt"
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5825U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 26 rows, 30 columns and 75 nonzeros
Model fingerprint: 0x793a5b42
Variable types: 0 continuous, 30 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 1e+06]
Presolve removed 18 rows and 15 columns
Presolve time: 0.00s
Presolved: 8 rows, 15 columns, 36 nonzeros
Variable types: 0 continuous, 15 integer (0 binary)
Found heuristic solution: objective 924476.83600

Root relaxation: objective 9.014740e+05, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incu

In [71]:
# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i in regions:
        for j in types_of_panels:
            print(f"Region: {i}, {j}: {x[i, j].x} panels, Build: {y[i, j].x}")
    print(f"Total Cost: ${model.objVal}")
else:
    print("No optimal solution found.")

Optimal solution found:
Region: Central Region, Monocrystalline: 619.0 panels, Build: 1.0
Region: Central Region, Polycrystalline: 995.0 panels, Build: 1.0
Region: Central Region, Thin-film: 0.0 panels, Build: 1.0
Region: East Region, Monocrystalline: 1.0 panels, Build: 1.0
Region: East Region, Polycrystalline: 525.0 panels, Build: 1.0
Region: East Region, Thin-film: 0.0 panels, Build: 1.0
Region: North East Region, Monocrystalline: 3.0 panels, Build: 1.0
Region: North East Region, Polycrystalline: 458.0 panels, Build: 1.0
Region: North East Region, Thin-film: 0.0 panels, Build: 1.0
Region: North Region, Monocrystalline: 4.0 panels, Build: 1.0
Region: North Region, Polycrystalline: 447.0 panels, Build: 1.0
Region: North Region, Thin-film: 0.0 panels, Build: 1.0
Region: West Region, Monocrystalline: 4.0 panels, Build: 1.0
Region: West Region, Polycrystalline: 379.0 panels, Build: 1.0
Region: West Region, Thin-film: 0.0 panels, Build: 1.0
Total Cost: $901663.4104999998


In [72]:
for constr in model.getConstrs():
    print(f"Constraint: {constr.ConstrName}, Expression: {model.getRow(constr)}, RHS: {constr.RHS}")

Constraint: demand_Central Region, Expression: 60.0 x[Central Region,Monocrystalline] + 52.0 x[Central Region,Polycrystalline] + 28.000000000000004 x[Central Region,Thin-film], RHS: 88877.8
Constraint: space_Central Region, Expression: 1.393545 x[Central Region,Monocrystalline] + 1.393545 x[Central Region,Polycrystalline] + 1.393545 x[Central Region,Thin-film], RHS: 2559.5
Constraint: install_Central Region_Monocrystalline, Expression: x[Central Region,Monocrystalline] + -1000.0 y[Central Region,Monocrystalline], RHS: 0.0
Constraint: install_Central Region_Polycrystalline, Expression: x[Central Region,Polycrystalline] + -1000.0 y[Central Region,Polycrystalline], RHS: 0.0
Constraint: install_Central Region_Thin-film, Expression: x[Central Region,Thin-film] + -1000.0 y[Central Region,Thin-film], RHS: 0.0
Constraint: demand_East Region, Expression: 60.0 x[East Region,Monocrystalline] + 52.0 x[East Region,Polycrystalline] + 28.000000000000004 x[East Region,Thin-film], RHS: 27358.1999999999